# Export mmdetection model to other format

## 1. Load mmdetection model

In [1]:
from mmdet.apis import init_detector
from mmdet.utils import register_all_modules
import torch

# 指定模型的配置文件和 checkpoint 文件路径
config_file = 'configs/yolo/yolov3_mobilenetv1.py'
checkpoint_file = 'work_dirs/yolov3_mobilenetv1/best_pascal_voc_mAP_epoch_25.pth'

#Register all modules in mmdet into the registries
register_all_modules()
# 若检测到有GPU则使用GPU
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# 根据配置文件和 checkpoint 文件构建模型
model = init_detector(config_file, checkpoint_file, device=device)
model.eval()

Loads checkpoint by local backend from path: work_dirs/yolov3_mobilenetv1/best_pascal_voc_mAP_epoch_25.pth


YOLOV3(
  (data_preprocessor): DetDataPreprocessor()
  (backbone): MobileNetV1(
    (layer1): Sequential(
      (0): Sequential(
        (0): Conv2d(3, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=3, bias=False)
        (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv2d(3, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU6(inplace=True)
        (6): Conv2d(4, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (7): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (8): ReLU6(inplace=True)
      )
      (1): Sequential(
        (0): Conv2d(4, 4, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=4, bias=False)
        (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=Tr

## 2. Export to onnx

In [2]:
# Let's create a dummy input tensor  
dummy_input = torch.randn(1, 3, 224, 224, requires_grad=True, device=device)  

# Export the model   
torch.onnx.export(model,         # model being run 
     dummy_input,       # model input (or a tuple for multiple inputs) 
     "./outputs/onnx/best_pascal_voc_mAP_epoch_12.onnx",       # where to save the model  
     export_params=True,  # store the trained parameter weights inside the model file 
     opset_version=11,    # the ONNX version to export the model to 
     do_constant_folding=True,  # whether to execute constant folding for optimization 
     input_names = ['modelInput'],   # the model's input names 
     output_names = ['modelOutput'], # the model's output names 
     dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
                            'modelOutput' : {0 : 'batch_size'}}) 
print(" ") 
print('Model has been converted to ONNX')

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

 
Model has been converted to ONNX


## 3. Export onnx to kmodel 

In [ ]:
import nncase
import onnxsim
import onnx
import os

def parse_model_input_output(model_file):
    onnx_model = onnx.load(model_file)
    input_all = [node.name for node in onnx_model.graph.input]
    input_initializer = [node.name for node in onnx_model.graph.initializer]
    input_names = list(set(input_all) - set(input_initializer))
    input_tensors = [node for node in onnx_model.graph.input if node.name in input_names]

    # input
    inputs = []
    for _, e in enumerate(input_tensors):
        onnx_type = e.type.tensor_type
        input_dict = {}
        input_dict['name'] = e.name
        input_dict['dtype'] = onnx.mapping.TENSOR_TYPE_TO_NP_TYPE[onnx_type.elem_type]
        input_dict['shape'] = [(i.dim_value if i.dim_value != 0 else d) for i, d in zip(
            onnx_type.shape.dim, [1, 3, 224, 224])]
        inputs.append(input_dict)

    return onnx_model, inputs

def onnx_simplify(model_file):
    onnx_model, inputs = parse_model_input_output(model_file)
    onnx_model = onnx.shape_inference.infer_shapes(onnx_model)
    input_shapes = {}
    for input in inputs:
        input_shapes[input['name']] = input['shape']

    onnx_model, check = onnxsim.simplify(onnx_model, overwrite_input_shapes=input_shapes)
    assert check, "Simplified ONNX model could not be validated"

    model_file = os.path.join(os.path.dirname(model_file), 'best_pascal_voc_mAP_epoch_11_simplified.onnx')
    onnx.save_model(onnx_model, model_file)
    return model_file

def read_model_file(model_file):
    with open(model_file, 'rb') as f:
        model_content = f.read()
    return model_content

# onnx simplify
model_file = onnx_simplify("./outputs/onnx/best_pascal_voc_mAP_epoch_11.onnx")

# compile_options
compile_options = nncase.CompileOptions()
compile_options.target = 'k210'
compile_options.dump_ir = True
compile_options.dump_asm = True
compile_options.dump_dir = 'tmp'

# compiler
compiler = nncase.Compiler(compile_options)

# import_options
import_options = nncase.ImportOptions()

# import
model_content = read_model_file(model_file)
compiler.import_onnx(model_content, import_options)

# compile
compiler.compile()

# kmodel
kmodel = compiler.gencode_tobytes()
name = os.path.basename(model_file).split(".")[0]
with open(f'{name}.kmodel', 'wb') as f:
    f.write(kmodel)

## 4. Export to tflite

In [2]:
import os
from tinynn.converter import TFLiteConverter

dummy_input = torch.rand((1, 3, 224, 224))

output_path = os.path.join('./outputs/tflite/k210_yolo_v2.tflite')

# When converting quantized models, please ensure the quantization backend is set.
#torch.backends.quantized.engine = 'qnnpack'

# The code section below is used to convert the model to the TFLite format
# If you want perform dynamic quantization on the float models,
# you may refer to `dynamic.py`, which is in the same folder.
# As for static quantization (e.g. quantization-aware training and post-training quantization),
# please refer to the code examples in the `examples/quantization` folder.
converter = TFLiteConverter(model, dummy_input, output_path)
converter.convert()

INFO (tinynn.converter.base) Generated model saved to ./outputs/tflite/k210_tflite_v2.tflite


## 5. Export onnx to tflite (experimental)

In [ ]:
from onnx_tf.backend import prepare
import tensorflow as tf

TF_PATH = "./outputs/pb/best_pascal_voc_mAP_epoch_135.pb" # where the representation of tensorflow model will be stored
ONNX_PATH = "./outputs/onnx/best_pascal_voc_mAP_epoch_135_simplified.onnx" # path to my existing ONNX model
onnx_model = onnx.load(ONNX_PATH)  # load onnx model
tf_rep = prepare(onnx_model)  # creating TensorflowRep object
tf_rep.export_graph(TF_PATH)

TFLITE_PATH = "./outputs/pb/best_pascal_voc_mAP_epoch_135.tflite"
converter = tf.lite.TFLiteConverter.from_saved_model(TF_PATH)
## 启用量化配置
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
tf_lite_model = converter.convert()
with open(TFLITE_PATH, 'wb') as f:
    f.write(tf_lite_model)